In [ ]:
%%bash
pip install -U byzerllm
ray start --head

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


Error while terminating subprocess (pid=75420): 


2024-06-14 11:09:17,446	INFO usage_lib.py:472 -- Usage stats collection is enabled. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.
2024-06-14 11:09:17,446	INFO scripts.py:764 -- Local node IP: 127.0.0.1


Traceback (most recent call last):
  File "/opt/miniconda3/envs/byzerllm/bin/ray", line 8, in <module>
    sys.exit(main())
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/ray/scripts/scripts.py", line 2612, in main
    return cli()
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/opt/mi

In [ ]:
byzerllm deploy --pretrained_model_type saas/openai \
--cpus_per_worker 0.001 \
--gpus_per_worker 0 \
--worker_concurrency 10 \
--num_workers 1 \
--infer_params saas.base_url="https://api.deepseek.com/v1" saas.api_key=${MODEL_DEEPSEEK_TOKEN} saas.model=deepseek-chat \
--model deepseek_chat

In [1]:
import byzerllm

llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")

@byzerllm.prompt(llm = llm )
def hello(q:str) ->str:
    '''
    你好, {{ q }}
    '''

hello("你是谁")    

2024-06-14 15:06:16.833 | INFO     | byzerllm.utils.connect_ray:connect_cluster:37 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-06-14 15:06:16,893	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-06-14 15:06:16,919	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


'你好！我是一个人工智能助手，旨在提供信息查询、建议和解答问题等服务。如果你有任何问题或需要帮助，请随时告诉我。'

In [9]:
hello.prompt("你是谁")

'\n你好, 你是谁'

In [10]:
hello.with_llm(llm).run("你是谁")

'你好！我是一个人工智能助手，专门设计来回答问题、提供信息和帮助解决问题。如果你有任何疑问或需要帮助，请随时告诉我。'

In [11]:
import byzerllm
from byzerllm import ByzerLLM

llm = ByzerLLM.from_default_model("deepseek_chat")

@byzerllm.prompt()
def tell_story() -> str:
    """
    讲一个100字的故事。    
    """


s = (
    tell_story.with_llm(llm)
    .with_response_markers()
    .options({"llm_config": {"max_length": 10}})
    .run()
)
print(s)

2024-06-14 14:42:04.246 | INFO     | byzerllm.utils.connect_ray:connect_cluster:37 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-06-14 14:42:04,411	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-06-14 14:42:04,415	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.



从前，森林里住着一只聪明的小狐狸。一天，它发现了一块闪闪发光的宝石。小狐狸决定用这块宝石帮助森林里的动物们。它用宝石的光芒指引迷路的小鹿找到了回家的路，用宝石的温暖治愈了受伤的小鸟。从此，小狐狸成了森林里的英雄，动物们都感激它的善良和智慧。


In [12]:
import pydantic 

class Story(pydantic.BaseModel):
    '''
    故事
    '''

    title: str = pydantic.Field(description="故事的标题")
    body: str = pydantic.Field(description="故事主体")

@byzerllm.prompt()
def tell_story()->Story:
    '''
    讲一个100字的故事。    
    '''

s = tell_story.with_llm(llm).run()
print(isinstance(s, Story))
print(s.title)


True
勇敢的小鸟


In [16]:
from loguru import logger
import re

@byzerllm.prompt()
def generate_regex_pattern(desc: str) -> str:
    """
    根据下面的描述生成一个正则表达式，要符合python re.compile 库的要求。

    {{ desc }}

    最后生成的正则表达式要在<REGEX></REGEX>标签对里。
    """    

def extract_regex_pattern(regex_block: str) -> str:    
    pattern = re.search(r"<REGEX>(.*)</REGEX>", regex_block, re.DOTALL)
    if pattern is None:
        logger.warning("No regex pattern found in the generated block:\n {regex_block}")
        raise None
    return pattern.group(1)

pattern = "匹配一个邮箱地址"
v = generate_regex_pattern.with_llm(llm).with_extractor(extract_regex_pattern).run(desc=pattern)
print(v)


^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$



In [8]:
import byzerllm
data = {
    'name': 'Jane Doe',
    'task_count': 3,
    'tasks': [
        {'name': 'Submit report', 'due_date': '2024-03-10'},
        {'name': 'Finish project', 'due_date': '2024-03-15'},
        {'name': 'Reply to emails', 'due_date': '2024-03-08'}
    ]
}


class RAG():
    def __init__(self):        
        self.llm = byzerllm.ByzerLLM()
        self.llm.setup_template(model="deepseek_chat",template="auto")
        self.llm.setup_default_model_name("deepseek_chat")        
    
    @byzerllm.prompt(lambda self:self.llm)
    def generate_answer(self,name,task_count,tasks)->str:
        '''
        Hello {{ name }},

        This is a reminder that you have {{ task_count }} pending tasks:
        {% for task in tasks %}
        - Task: {{ task.name }} | Due: {{ task.due_date }}
        {% endfor %}

        Best regards,
        Your Reminder System
        '''        

t = RAG()

response = t.generate_answer(**data)
print(response)

Hello Jane Doe,

I hope this message finds you well. I wanted to remind you of your 3 pending tasks to ensure you stay on track:

1. **Submit report** - This task is due on **2024-03-10**. Please ensure that you allocate time to complete and submit the report before the deadline.

2. **Finish project** - The deadline for this project is **2024-03-15**. It's important to keep an eye on your progress and make adjustments as necessary to meet this due date.

3. **Reply to emails** - This task is due soonest, on **2024-03-08**. Responding to emails promptly is crucial for maintaining good communication and relationships.

Please prioritize these tasks according to their urgency and importance. If you need any assistance or have any questions, feel free to reach out.

Best regards,
Your Reminder System


In [17]:
import byzerllm

data = {
    'name': 'Jane Doe',
    'task_count': 3,
    'tasks': [
        {'name': 'Submit report', 'due_date': '2024-03-10'},
        {'name': 'Finish project', 'due_date': '2024-03-15'},
        {'name': 'Reply to emails', 'due_date': '2024-03-08'}
    ]
}


class RAG():
    def __init__(self):        
        self.llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
    
    @byzerllm.prompt()
    def generate_answer(self,name,task_count,tasks)->str:
        '''
        Hello {{ name }},

        This is a reminder that you have {{ task_count }} pending tasks:
            
        {{ tasks }}

        Best regards,
        Your Reminder System
        '''
        
        tasks_str = "\n".join([f"- Task: {task['name']} | Due: { task['due_date'] }" for task in tasks])
        return {"tasks": tasks_str}

t = RAG()

response = t.generate_answer.with_llm(t.llm).run(**data)
print(response)

2024-06-14 14:59:25.434 | INFO     | byzerllm.utils.connect_ray:connect_cluster:37 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-06-14 14:59:25,508	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-06-14 14:59:25,509	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.


Hello Jane Doe,

Just a gentle nudge to keep you on track with your pending tasks. Here's a quick recap:

1. **Submit report** - Your deadline is approaching on **2024-03-10**. Ensure all data is accurate and the report is comprehensive before submission.

2. **Finish project** - Aim to wrap up your project by **2024-03-15**. Double-check all components and ensure they meet the project's objectives.

3. **Reply to emails** - Time-sensitive responses are needed by **2024-03-08**. Prioritize urgent emails and maintain clear communication.

Remember, staying ahead of these deadlines will help maintain your productivity and efficiency. If you need any assistance or have questions, feel free to reach out.

Best regards,
Your Reminder System


In [2]:
import pydantic
class Time(pydantic.BaseModel):
    time: str = pydantic.Field(...,description="时间，时间格式为 yyyy-MM-dd")


@llm.impl()
def calculate_current_time()->Time:
    '''
    计算当前时间
    '''
    pass 


calculate_current_time()
#output: Time(time='2024-06-14')

Time(time='2024-06-14')

In [2]:
import byzerllm

llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")

v = llm.stream_chat_oai(model="deepseek_chat",conversations=[{
    "role":"user",
    "content":"你好，你是谁",
}],delta_mode=True)

for t in v:
    print(t[0],flush=True)  

2024-06-14 15:12:21.200 | INFO     | byzerllm.utils.connect_ray:connect_cluster:37 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-06-14 15:12:21,271	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-06-14 15:12:21,272	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.



你好
！
我
是一个
人工智能
助手
，
旨在
提供
信息
、
解答
问题
和
帮助
用户
解决问题
。
如果你
有
任何
疑问
或
需要
帮助
，
请
随时
告诉我
。


In [4]:
import byzerllm

llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")

v = llm.chat_oai(model="deepseek_chat",conversations=[{
    "role":"user",
    "content":"你好，你是谁",
}])

print(v[0].output)

2024-06-14 15:14:33.450 | INFO     | byzerllm.utils.connect_ray:connect_cluster:37 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-06-14 15:14:33,546	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-06-14 15:14:33,547	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.


你好！我是一个人工智能助手，旨在提供信息、解答问题和帮助用户解决问题。如果你有任何问题或需要帮助，请随时告诉我。


In [5]:
from typing import List,Dict,Any,Annotated
import pydantic 
import datetime
from dateutil.relativedelta import relativedelta

def compute_date_range(count:Annotated[int,"时间跨度，数值类型"],
                       unit:Annotated[str,"时间单位，字符串类型",{"enum":["day","week","month","year"]}])->List[str]:
    '''
    计算日期范围

    Args:
        count: 时间跨度，数值类型
        unit: 时间单位，字符串类型，可选值为 day,week,month,year
    '''        
    now = datetime.datetime.now()
    now_str = now.strftime("%Y-%m-%d %H:%M:%S")
    if unit == "day":
        return [(now - relativedelta(days=count)).strftime("%Y-%m-%d %H:%M:%S"),now_str]
    elif unit == "week":
        return [(now - relativedelta(weeks=count)).strftime("%Y-%m-%d %H:%M:%S"),now_str]
    elif unit == "month":
        return [(now - relativedelta(months=count)).strftime("%Y-%m-%d %H:%M:%S"),now_str]
    elif unit == "year":
        return [(now - relativedelta(years=count)).strftime("%Y-%m-%d %H:%M:%S"),now_str]
    return ["",""]

def compute_now()->str:
    '''
    计算当前时间
    '''
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [6]:
t = llm.chat_oai([{
    "content":'''计算当前时间''',
    "role":"user"    
}],tools=[compute_date_range,compute_now],execute_tool=True)

t[0].values

['2024-06-14 15:18:02']

In [7]:
t = llm.chat_oai([{
    "content":'''最近三个月趋势''',
    "role":"user"    
}],tools=[compute_date_range,compute_now],execute_tool=True)

t[0].values

[['2024-03-14 15:19:13', '2024-06-14 15:19:13']]